In [1]:
import pandas as pd
import tweepy
from textblob import TextBlob
from wordcloud import WordCloud
import numpy as np
import matplotlib.pyplot as plt
import re

In [2]:
consumer_key = "CSuIiOnVjfkIVlwoOt8TmrGW7"
consumer_key_secret = "wz0VqziOsFNr1vaz1RY3XySNHajIeW7D2SKdyMP76Z89rJuHvd"
access_token = "1457745168690450437-7dnKuIzrBMIxTJSqQ0eSfaNycyp7BV"
access_token_secret = "mfkismanuwheyKmCsXCbFIVLNibt2MW8CZas3qfqFPken"

In [3]:
auth = tweepy .OAuthHandler(consumer_key, consumer_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [4]:
def cleanText(text):
    text = re.sub(r"@[A-Za-z0-9]+", '', text) #removed @mentions
    text = re.sub(r'#', '', text) #removing # symbol
    text = re.sub(r'RT[\s]+', '', text) #removing retweets RT
    text = re.sub(r'https?:\/\/\S+', '', text) #removes the hyperlink

    return text

def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

def getPolarity(text):
    return TextBlob(text).sentiment.polarity

def getAnalysis(score):
    if score < 0:
        return "Negative"
    elif score == 0:
        return "Neutral"
    else:
        return "Positive"

In [5]:
def get_keyword_tweets(keyword, count):
    items = api.search_30_day(label='Development', query=keyword, maxResults= count)
    return items

In [7]:
p = get_keyword_tweets("bill gates", 10)
p[0]._json

{'created_at': 'Mon Nov 15 09:18:46 +0000 2021',
 'id': 1460175433585025026,
 'id_str': '1460175433585025026',
 'text': 'Grapperhaus is volledig door de mand gevallen maar natuurlijk wordt hij 100% gesteund door georganiseerde Bill Gate… https://t.co/OIkMCtXIin',
 'source': '<a href="http://www.realisteninverzet.nl" rel="nofollow">Realistbot NL</a>',
 'truncated': True,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 1057191992818577408,
  'id_str': '1057191992818577408',
  'name': 'Realistbot 🚜🇱🇺',
  'screen_name': 'RealistBotNL',
  'location': 'HOLLAND ME VADERLAND 🇱🇺',
  'url': 'https://bit.ly/2HePvIg',
  'description': 'pro europa dus anti eussr!! pvv op 1! nederland is een DICTATUUR ONDER RUTTE #oprutte #realist #verzet #boeren #parodie #antiantifa! broertje van @WokeBotNL',
  'translator_type': 'none',
  'protected': False,
  'verified': False,
  '

In [6]:
def get_new_table(stockname):
    answer = get_keyword_tweets(stockname, 100)
    df = pd.DataFrame({"tweet": [tweet.text for tweet in answer], "language":[tweet.lang for tweet in answer],"likes": [tweet.favorite_count for tweet in answer],"retweets": [tweet.retweet_count for tweet in answer], "datetime":[tweet.created_at for tweet in answer]})
    df = df[df.language == "en"]
    df["Subjectivity"] = df["tweet"].apply(getSubjectivity)
    df["Polarity"] = df["tweet"].apply(getPolarity)
    df["Analysis"] = df["Polarity"].apply(getAnalysis)
    return df

In [8]:
infy_csv = get_new_table("INFOSYS")
infy_csv.to_csv("infy.csv", encoding="utf-8")

In [9]:
tata_df = get_new_table("TATASTEEL")
tata_df.to_csv('tata.csv', mode='a', index=False, header=False)